In [1]:
%cd /hdd/yuchen

/hdd/yuchen


In [2]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import torchtext
import nltk, re, string, collections
from nltk.util import ngrams
import collections
import scipy 
import IProgress 
from ipywidgets import FloatProgress

%matplotlib inline
BATCH_SIZE = 128
EPOCH = 5

In [3]:
featname = 'c'
use_stim = 'each'

subject= 'subj01'
use_stim = 'each'
topdir = '/hdd/yuchen/nsdfeat'

savedir = f'{topdir}/subjfeat/'
featdir = f'{topdir}/15dim/{featname}/'

nsd_expdesign = scipy.io.loadmat('/hdd/yuchen/nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

# Note that most of them are 1-base index!
sharedix = nsd_expdesign['sharedix'] -1 
stims = np.load(f'/hdd/yuchen/mrifeat/{subject}/{subject}_stims.npy')

feats = []
tr_idx = np.zeros(len(stims))

ct = 0

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    if os.path.exists(f'{featdir}/{s:06}.npy'):
        feat = np.load(f'{featdir}/{s:06}.npy')
        feats.append(feat)
    else: 
        # print(f'{s:06}.npy does not exist')
        feats.append(np.array([0]*768*15))
        ct += 1

feats = np.stack(feats)    

feats_tr = feats[tr_idx==1]
feats_te = feats[tr_idx==0]

0it [00:00, ?it/s]

In [4]:
feats_tr.shape, feats_te.shape

((24980, 11520), (2770, 11520))

In [5]:
savedir = '/hdd/yuchen/mrifeat/subj01/'

betas_tr = np.load(f'{savedir}/{subject}_ventral_betas_tr.npy')
betas_te = np.load(f'{savedir}/{subject}_ventral_betas_te.npy')
betas_tr_avg = np.load(f'{savedir}/{subject}_ventral_betas_ave_tr.npy')
betas_te_avg = np.load(f'{savedir}/{subject}_ventral_betas_ave_te.npy')

In [6]:
betas_tr.shape, betas_te.shape

((24980, 7604), (2770, 7604))

In [7]:
empty_idx = np.where(np.mean(feats_tr,axis=1) == 0.0)[0]
mask = np.ones(len(feats_tr), dtype=bool) 
mask[empty_idx] = False  
feats_tr = feats_tr[mask]
betas_tr = betas_tr[mask]

empty_idx = np.where(np.mean(feats_te,axis=1) == 0.0)[0]
mask = np.ones(len(feats_te), dtype=bool) 
mask[empty_idx] = False  
feats_te = feats_te[mask]
betas_te = betas_te[mask]

In [8]:
stims = np.load(f'/hdd/yuchen/mrifeat/{subject}/{subject}_stims_ave.npy')

feats_avg = []
tr_idx = np.zeros(len(stims))

ct = 0

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    if os.path.exists(f'{featdir}/{s:06}.npy'):
        # feat = np.load(f'{featdir}/{s:06}.npy')[0]
        feat = np.load(f'{featdir}/{s:06}.npy')
        
        feats_avg.append(feat)
    else: 
        # print(f'{s:06}.npy does not exist')
        # feats_avg.append('empty')
        feats_avg.append(np.array([0]*768*15))
        ct += 1

feats_avg = np.stack(feats_avg)    

feats_tr_avg = feats_avg[tr_idx==1]
feats_te_avg = feats_avg[tr_idx==0]

0it [00:00, ?it/s]

In [9]:
feats_tr_avg.shape, feats_te_avg.shape

((8859, 11520), (982, 11520))

In [10]:
empty_idx = np.where(np.mean(feats_tr_avg,axis=1) == 0.0)[0]
mask = np.ones(len(feats_tr_avg), dtype=bool) 
mask[empty_idx] = False  
feats_tr_avg = feats_tr_avg[mask]
betas_tr_avg = betas_tr_avg[mask]

empty_idx = np.where(np.mean(feats_te_avg,axis=1) == 0.0)[0]
mask = np.ones(len(feats_te_avg), dtype=bool) 
mask[empty_idx] = False  
feats_te_avg = feats_te_avg[mask]
betas_te_avg = betas_te_avg[mask]

In [11]:
feats_tr_avg.shape, feats_te_avg.shape, betas_tr_avg.shape, betas_te_avg.shape

((8859, 11520), (982, 11520), (8859, 7604), (982, 7604))

In [12]:
feats_tr.shape, betas_tr.shape

((24980, 11520), (24980, 7604))

In [147]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [148]:
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

In [149]:
X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.7283405637940131

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, betas, feats):
        betas = (betas - betas.mean()) / betas.std()
        self.betas = torch.tensor(betas, dtype=torch.float32)
        feats = feats.reshape(-1,15,768)
        feats = feats[:,1:,:].reshape(-1,14*768)
        self.feats = torch.tensor(feats, dtype=torch.float32)

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, idx):
        return {'betas': self.betas[idx], 'feats': self.feats[idx]}

train_dataset = CustomDataset(betas_tr, feats_tr)
test_dataset = CustomDataset(betas_te_avg, feats_te_avg)

In [14]:
cls = feats_tr[0,:768]
cls.shape

(768,)

In [15]:
batch_size = 16  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    break
num_neurons = batch['betas'].shape[-1]
num_features = batch['feats'].shape[-1]

## original text encoder

In [16]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, num_neurons, num_features):
        super(NeuralNetwork, self).__init__()
        self.num_neurons = num_neurons
        channel = 64
        
        self.conv1d = nn.Conv1d(1, channel, kernel_size=1)
        self.relu = nn.ReLU()
        
        self.net = nn.Sequential(
            nn.Conv1d(channel, channel, kernel_size=7, padding = 3),
            nn.ReLU(),
            nn.Conv1d(channel, channel, kernel_size=7, padding = 3),
            nn.ReLU(),
            nn.Conv1d(channel, channel, kernel_size=7, padding=3),
        )

        self.downsample = nn.Conv1d(channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(num_neurons, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 14*768)
        )

    def forward(self, x):
        x = self.conv1d(x)
        for i in range(3):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.reshape(-1, self.num_neurons)
        x = self.fc(x)
        # x = x.reshape(-1, 14, 768)
        # x = x.reshape(-1, 14*768)
        return x



In [17]:
device = 'cuda:1'
model = NeuralNetwork(num_neurons = num_neurons, num_features = num_features).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        betas = batch['betas'].to(device).unsqueeze(1)
        feats = batch['feats'].to(device)
        
        pred = model(betas)
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            betas = batch['betas'].to(device).unsqueeze(1)
            feats = batch['feats'].to(device)
            pred = model(betas)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.32772 val loss 0.31585
[epoch 1] train loss 0.29593 val loss 0.29396
[epoch 2] train loss 0.28134 val loss 0.27687
[epoch 3] train loss 0.26775 val loss 0.27569
[epoch 4] train loss 0.25213 val loss 0.26719
[epoch 5] train loss 0.23392 val loss 0.26564
[epoch 6] train loss 0.21524 val loss 0.26937
[epoch 7] train loss 0.19865 val loss 0.26884
[epoch 8] train loss 0.18486 val loss 0.27259
[epoch 9] train loss 0.17349 val loss 0.27485
[epoch 10] train loss 0.16427 val loss 0.2737
[epoch 11] train loss 0.15627 val loss 0.27388
end


In [23]:
model.load_state_dict(torch.load('/hdd/yuchen/modelweights.pth'))

<All keys matched successfully>

In [24]:
lst = []
lst_gt = []
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        betas = batch['betas'].to(device).unsqueeze(1)
        feats = batch['feats'].to(device)
        pred = model(betas)
        lst.append(pred.cpu().detach().numpy())
        lst_gt.append(feats.cpu().detach().numpy())
    

In [25]:
lst = np.vstack(lst)
lst_gt = np.vstack(lst_gt)

In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(lst,lst_gt)

0.26555938

In [27]:
lst = lst.reshape(-1, 14, 768)
lst_gt = lst_gt.reshape(-1, 14, 768)
cls_ = np.tile(cls, (982, 1))
cls_ = np.expand_dims(cls_, 1)
lst = np.concatenate((cls_, lst), axis=1)
lst_gt = np.concatenate((cls_, lst_gt), axis=1)
np.save('/hdd/yuchen/temp_lst.npy', lst)
np.save('/hdd/yuchen/temp_lst_gt.npy', lst_gt)